# Amazon Bedrock AgentCore Runtime에서 Bedrock Manus 호스팅하기 (VPC Private 연결)

이 튜토리얼에서는 **VPC Endpoint를 통한 Private 연결**을 사용하여 Amazon Bedrock AgentCore Runtime에서 Bedrock Manus 에이전트를 호스팅하는 방법을 배웁니다.

### 🔒 VPC Private 연결의 장점

- ✅ **완전한 Private 통신**: AgentCore ↔ Fargate 간 Public 인터넷 사용 없음
- ✅ **보안 강화**: Public IP 노출 제거, VPC Endpoint 통해서만 접근
- ✅ **Zero Trust**: Security Group 간 참조를 통한 최소 권한 원칙
- ✅ **규정 준수**: 데이터가 AWS 네트워크 내부에서만 전송

### Overview

| 정보                | 세부사항                                                                      |
|:--------------------|:-----------------------------------------------------------------------------|
| 에이전트 유형        | 멀티 에이전트 (Planner, Supervisor, Coder, Reporter)                         |
| 에이전틱 프레임워크   | Strands                                                                      |
| LLM 모델            | Anthropic Claude Sonnet 3.7                                                  |
| **네트워크 설정**    | **VPC Endpoint (Private 연결)** 🔒                                           |
| 튜토리얼 구성요소    | AgentCore Runtime에서 에이전트 호스팅. VPC Private 연결 사용                   |
| 튜토리얼 분야        | 범용                                                                         |
| 예제 복잡도         | 중급                                                                         |
| 사용된 SDK          | Amazon BedrockAgentCore Python SDK 및 boto3                                 |

### 📋 목차

#### 사전 준비: VPC 인프라 확인
- VPC Endpoint 상태 확인
- Internal ALB 확인
- Test VPC 설정 로드

#### 1) AgentCore Runtime 배포 (VPC Private 연결)
- **Step 1.** IAM Role 설정
- **Step 2.** VPC 설정 추가
- **Step 3.** 배포 구성: `agentcore_runtime.configure()`
- **Step 4.** 배포: `agentcore_runtime.launch()`
- **Step 5.** 상태 확인: `agentcore_runtime.status()`

#### 2) 배포한 에이전트 호출 및 검증
- Private IP를 통한 호출
- 로그 분석 (VPC CIDR 확인)
- 성능 측정


## 사전 준비사항

### 필수 요구사항
* Python 3.11+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Docker running

### VPC 인프라 요구사항 🔒
* **Test VPC 구축 완료** (`test_vpc_setup_new_vpc.sh` 실행 완료)
* VPC Endpoints (Bedrock AgentCore)
* Internal ALB (internet-facing 아님)
* Private Subnets
* Security Groups 설정 완료

### ⚠️ 중요 발견사항 (2025-10-18)

**지원되는 Availability Zone**:
- ✅ **use1-az2** (us-east-1a): subnet-0b2fb367d6e823a79 - 지원됨
- ❌ **use1-az6** (us-east-1c): subnet-0ed3a6040386768cf - **지원 안 됨!**

AWS 공식 문서에 따르면, us-east-1 리전에서 Bedrock AgentCore는 다음 AZ만 지원합니다:
- use1-az1, use1-az2, use1-az4

**이 노트북은 지원되는 단일 Subnet (use1-az2)만 사용하도록 수정되었습니다.**

**Observability 필수**:
- CloudWatch 로그 없이는 health check 실패 원인을 디버깅할 수 없습니다
- 이 노트북은 observability를 자동으로 활성화합니다

### VPC 인프라 확인

아직 VPC 인프라를 구축하지 않았다면:
```bash
cd test_vpc_private_setup
./test_vpc_setup_new_vpc.sh
python3 test_vpc_private_connectivity.py
```

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import sys, os
module_path = "../../../.."
sys.path.append(os.path.abspath(module_path))

# 사전 준비: VPC 인프라 확인

AgentCore Runtime을 VPC에 배포하기 전에, Test VPC 인프라가 올바르게 구축되었는지 확인합니다.

In [15]:
import json

# Test VPC 설정 로드
with open('test_vpc_config.json', 'r') as f:
    vpc_config = json.load(f)

print("=" * 60)
print("Test VPC Configuration")
print("=" * 60)
print(f"VPC ID: {vpc_config['vpc_id']}")
print(f"VPC CIDR: {vpc_config['vpc_cidr']}")
print(f"Region: {vpc_config['region']}")
print()
print("Private Subnets:")
for subnet in vpc_config['subnets']['private']:
    print(f"  - {subnet}")
print()
print("VPC Endpoints:")
print(f"  Data Plane: {vpc_config['vpc_endpoints']['data_plane']}")
print(f"  Gateway: {vpc_config['vpc_endpoints']['gateway']}")
print()
print("Security Groups:")
for sg_name, sg_id in vpc_config['security_groups'].items():
    print(f"  {sg_name}: {sg_id}")
print()
print("Internal ALB:")
print(f"  DNS: {vpc_config['alb']['dns']}")
print(f"  ARN: {vpc_config['alb']['arn']}")
print("=" * 60)

Test VPC Configuration
VPC ID: vpc-05975448296a22c21
VPC CIDR: 10.100.0.0/16
Region: us-east-1

Private Subnets:
  - subnet-0b2fb367d6e823a79
  - subnet-0ed3a6040386768cf

VPC Endpoints:
  Data Plane: vpce-0b1c05f284838d951
  Gateway: vpce-00259de820f493d28

Security Groups:
  agentcore: sg-0affaea9ac4dc26b1
  alb: sg-061896ca7967f6183
  fargate: sg-0e1314a2421686c2c
  vpce: sg-085cf66da6c4027d2

Internal ALB:
  DNS: internal-test-vpc-private-alb-1154687901.us-east-1.elb.amazonaws.com
  ARN: arn:aws:elasticloadbalancing:us-east-1:057716757052:loadbalancer/app/test-vpc-private-alb/e15d1372ab65062e


### VPC Endpoint 상태 확인

In [16]:
import boto3

ec2_client = boto3.client('ec2', region_name=vpc_config['region'])

# VPC Endpoint 상태 확인
vpce_ids = [
    vpc_config['vpc_endpoints']['data_plane'],
    vpc_config['vpc_endpoints']['gateway']
]

response = ec2_client.describe_vpc_endpoints(VpcEndpointIds=vpce_ids)

print("VPC Endpoint Status:")
for endpoint in response['VpcEndpoints']:
    endpoint_id = endpoint['VpcEndpointId']
    state = endpoint['State']
    service_name = endpoint['ServiceName']
    
    status_icon = "✅" if state == 'available' else "⚠️"
    print(f"{status_icon} {endpoint_id}")
    print(f"   Service: {service_name}")
    print(f"   State: {state}")
    print()

all_available = all(ep['State'] == 'available' for ep in response['VpcEndpoints'])
if all_available:
    print("✅ All VPC Endpoints are available!")
else:
    print("⚠️  Some VPC Endpoints are not available. Please check the status.")

VPC Endpoint Status:
✅ vpce-0b1c05f284838d951
   Service: com.amazonaws.us-east-1.bedrock-agentcore
   State: available

✅ vpce-00259de820f493d28
   Service: com.amazonaws.us-east-1.bedrock-agentcore.gateway
   State: available

✅ All VPC Endpoints are available!


---

# 1) AgentCore Runtime 배포 (VPC Private 연결)

이제 Bedrock Manus를 AgentCore Runtime에 배포하되, **VPC Endpoint를 통한 Private 연결**을 사용하겠습니다.

## Step 1. IAM Role 설정

In [17]:
# IAM Role 설정 (기존과 동일)
import sys
import os

current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))
utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.join(utils_dir, '..')
utils_dir = os.path.abspath(utils_dir)

sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

agent_name = "bedrock_manus_runtime_vpc"

# IAM Role (기존에 생성된 role 사용)
agentcore_iam_role = {
    'Role': {
        'Path': '/',
        'RoleName': 'agentcore-bedrock_manus_runtime-role',
        'RoleId': 'AROAQ24BP2I6GBFPY65CS',
        'Arn': 'arn:aws:iam::057716757052:role/agentcore-bedrock_manus_runtime-role',
        'CreateDate': '2025-09-28T01:33:47Z',
        'AssumeRolePolicyDocument': {
            'Version': '2012-10-17',
            'Statement': [{
                'Sid': 'AssumeRolePolicy',
                'Effect': 'Allow',
                'Principal': {'Service': 'bedrock-agentcore.amazonaws.com'},
                'Action': 'sts:AssumeRole',
                'Condition': {
                    'StringEquals': {'aws:SourceAccount': '057716757052'},
                    'ArnLike': {'aws:SourceArn': 'arn:aws:bedrock-agentcore:us-east-1:057716757052:*'}
                }
            }]
        }
    }
}

print(f"Using IAM Role: {agentcore_iam_role['Role']['Arn']}")

sys.path[0]: /home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus
Using IAM Role: arn:aws:iam::057716757052:role/agentcore-bedrock_manus_runtime-role


## Step 2. VPC 설정 추가 🔒

### VPC Configuration

AgentCore Runtime을 VPC에 배포하기 위해 다음 정보가 필요합니다:

1. **Subnet IDs**: Private subnet에 runtime을 배포
2. **Security Group IDs**: AgentCore용 Security Group
3. **VPC Endpoint**: Bedrock AgentCore VPC Endpoint (자동으로 사용됨)

### 네트워크 흐름

```
Bedrock AgentCore (AWS)
    ↓ Private (PrivateLink)
VPC Endpoint (10.100.x.x)
    ↓ Private
Internal ALB (10.100.x.x)
    ↓ Private
Fargate Runtime (10.100.x.x)
```

In [18]:
# VPC 설정 준비
# ⚠️ 중요: use1-az6 (subnet-0ed3a6040386768cf)는 지원되지 않음!
# AWS 공식 문서: us-east-1에서 use1-az1, use1-az2, use1-az4만 지원

# 지원되는 Subnet만 사용 (use1-az2)
supported_subnets = ['subnet-0b2fb367d6e823a79']  # us-east-1a (use1-az2) ✅

vpc_configuration = {
    'subnetIds': supported_subnets,
    'securityGroupIds': [vpc_config['security_groups']['agentcore']]
}

print("=" * 60)
print("VPC Configuration for AgentCore Runtime")
print("=" * 60)
print(f"Subnets (Private): {vpc_configuration['subnetIds']}")
print(f"Security Groups: {vpc_configuration['securityGroupIds']}")
print()
print("📝 Note:")
print("  - Using ONLY supported AZ (use1-az2)")
print("  - Subnet use1-az6 is NOT supported and excluded")
print("  - Runtime will be deployed in PRIVATE subnet")
print("  - No public IP will be assigned")
print("  - Communication via VPC Endpoint only")
print("=" * 60)

VPC Configuration for AgentCore Runtime
Subnets (Private): ['subnet-0b2fb367d6e823a79']
Security Groups: ['sg-0affaea9ac4dc26b1']

📝 Note:
  - Using ONLY supported AZ (use1-az2)
  - Subnet use1-az6 is NOT supported and excluded
  - Runtime will be deployed in PRIVATE subnet
  - No public IP will be assigned
  - Communication via VPC Endpoint only


## Step 3. 배포 구성: `agentcore_runtime.configure()`

### ⚠️ 중요: VPC 설정 추가

기존 `configure()` 메소드에서 달라진 점:
- **추가되지 않음**: VPC 설정은 `launch()` 또는 별도 API 호출 시 적용
- `configure()`는 Docker 이미지 및 entrypoint 설정만 담당

VPC 설정은 다음 단계에서 적용됩니다.

In [19]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print("region:", region)

agentcore_runtime = Runtime()

# Configure - 현재 디렉토리의 파일 사용
response = agentcore_runtime.configure(
    agent_name=agent_name,
    entrypoint="agentcore_runtime.py",  # 현재 디렉토리의 agentcore_runtime.py 사용
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",  # 현재 디렉토리의 requirements.txt 사용
    region=region
)

print()
print("✅ Configuration completed!")
print(f"   Config file: {response.config_path}")
print(f"   Dockerfile: {response.dockerfile_path}")
response

Entrypoint parsed: file=/home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/test_vpc_private_setup/agentcore_runtime.py, bedrock_agentcore_name=agentcore_runtime
Configuring BedrockAgentCore agent: bedrock_manus_runtime_vpc


region: us-east-1


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/test_vpc_private_setup/Dockerfile
Generated .dockerignore: /home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/test_vpc_private_setup/.dockerignore
Keeping 'bedrock_manus_runtime_vpc' as default agent
Bedrock AgentCore configured: /home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/test_vpc_private_setup/.bedrock_agentcore.yaml



✅ Configuration completed!
   Config file: /home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/test_vpc_private_setup/.bedrock_agentcore.yaml
   Dockerfile: /home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/test_vpc_private_setup/Dockerfile


ConfigureResult(config_path=PosixPath('/home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/test_vpc_private_setup/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/test_vpc_private_setup/Dockerfile'), dockerignore_path=PosixPath('/home/ubuntu/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/08_bedrock_manus/use_cases/05_insight_extractor_strands_sdk_workshop_phase_2/test_vpc_private_setup/.dockerignore'), runtime='Docker', region='us-east-1', account_id='057716757052', execution_role='arn:aws:iam::057716757052:role/agentcore-bedrock_manus_runtime-role', ecr_repository=None, auto_create_ecr=True)

### Configure 후 VPC 설정 적용

`.bedrock_agentcore.yaml` 파일에 VPC 설정을 추가합니다.

In [20]:
import yaml

# .bedrock_agentcore.yaml 파일 읽기
config_file_path = response.config_path
with open(config_file_path, 'r') as f:
    bedrock_config = yaml.safe_load(f)

# AWS API 형식에 맞춰 VPC 설정 추가
agent_name = 'bedrock_manus_runtime_vpc'
if agent_name in bedrock_config['agents']:
    agent_config = bedrock_config['agents'][agent_name]
    
    # VPC 네트워크 설정 (AWS API 형식)
    agent_config['aws']['network_configuration'] = {
        'network_mode': 'VPC',
        'network_mode_config': {
            'security_groups': vpc_configuration['securityGroupIds'],
            'subnets': vpc_configuration['subnetIds']
        }
    }
    
    # 🔥 CRITICAL: Observability 설정 추가 (CloudWatch 로그 활성화)
    agent_config['aws']['observability'] = {
        'enabled': True
    }

# 파일에 저장
with open(config_file_path, 'w') as f:
    yaml.dump(bedrock_config, f, default_flow_style=False, sort_keys=False)

print("✅ VPC configuration added to .bedrock_agentcore.yaml")
print()
print("Network Configuration:")
print(f"  Network Mode: VPC")
print(f"  Subnets: {vpc_configuration['subnetIds']}")
print(f"  Security Groups: {vpc_configuration['securityGroupIds']}")
print()
print("🔥 Observability Configuration:")
print(f"  CloudWatch Logs: ENABLED ✅")
print(f"  This will allow debugging via CloudWatch logs")
print()
print("📝 Note: VPC Endpoints will be used automatically when runtime is in VPC")

✅ VPC configuration added to .bedrock_agentcore.yaml

Network Configuration:
  Network Mode: VPC
  Subnets: ['subnet-0b2fb367d6e823a79']
  Security Groups: ['sg-0affaea9ac4dc26b1']

🔥 Observability Configuration:
  CloudWatch Logs: ENABLED ✅
  This will allow debugging via CloudWatch logs

📝 Note: VPC Endpoints will be used automatically when runtime is in VPC


## Step 4. 배포: AWS CLI로 Runtime 생성

### 🚀 VPC Private 배포 (SDK 버그 우회)

**문제**: `agentcore_runtime.launch()`는 VPC `network_mode_config`를 YAML에서 제거하는 버그가 있습니다.

**해결**: 기존 ECR 이미지를 사용하여 AWS CLI로 Runtime만 생성합니다.

이 방법으로:
- ✅ VPC 설정 보존 (security_groups, subnets)
- ✅ Private subnet에 배포
- ✅ Observability 활성화 (CloudWatch 로그)

In [27]:
# AWS CLI로 VPC Runtime 생성 (JSON 파일 사용)
import yaml
import subprocess
import json
import time
import tempfile

# YAML에서 설정 읽기
with open('.bedrock_agentcore.yaml', 'r') as f:
    config = yaml.safe_load(f)

agent_config = config['agents']['bedrock_manus_runtime_vpc']
network_config = agent_config['aws']['network_configuration']
account_id = agent_config['aws']['account']

# VPC 설정
subnets = network_config['network_mode_config']['subnets']
security_groups = network_config['network_mode_config']['security_groups']
execution_role = agent_config['aws']['execution_role']

# ECR repository - null이면 기존 PUBLIC runtime의 ECR 이미지 사용
ecr_repo = agent_config['aws']['ecr_repository']
if not ecr_repo or ecr_repo == 'null':
    ecr_repo = f"{account_id}.dkr.ecr.{region}.amazonaws.com/bedrock-agentcore-bedrock_manus_runtime"
    print(f"⚠️  Using PUBLIC runtime's ECR image: {ecr_repo}")

print("=" * 60)
print("Creating VPC Runtime with AWS CLI v2")
print("=" * 60)
print(f"ECR Image: {ecr_repo}:latest")
print(f"VPC Subnets: {subnets}")
print(f"Security Groups: {security_groups}")
print("=" * 60)
print()

# Runtime 생성 요청 JSON
runtime_name = f"bedrock_manus_runtime_vpc_{int(time.time())}"

create_request = {
    "agentRuntimeName": runtime_name,
    "agentRuntimeArtifact": {
        "containerConfiguration": {
            "containerUri": f"{ecr_repo}:latest"
        }
    },
    "roleArn": execution_role,
    "networkConfiguration": {
        "networkMode": "VPC",
        "networkModeConfig": {
            "securityGroups": security_groups,
            "subnets": subnets
        }
    },
    "protocolConfiguration": {
        "serverProtocol": "HTTP"
    }
}

# 임시 JSON 파일 생성
with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    json.dump(create_request, f, indent=2)
    json_file = f.name

print(f"🚀 Creating Runtime: {runtime_name}")
print()

try:
    # AWS CLI 실행 (이제 venv가 v2 사용)
    result = subprocess.run(
        ['aws', 'bedrock-agentcore-control', 'create-agent-runtime',
        '--cli-input-json', f'file://{json_file}',
        '--region', region],
        capture_output=True,
        text=True
    )

    if result.returncode == 0:
        response = json.loads(result.stdout)

        agent_arn = response['agentRuntimeArn']
        runtime_id = response['agentRuntimeId']
        status = response['status']

        print("✅ Runtime Created!")
        print(f"   ARN: {agent_arn}")
        print(f"   ID: {runtime_id}")
        print(f"   Status: {status}")
        print()
        print("ℹ️  Note: Runtime created in VPC mode with networkModeConfig")

        # launch_result 객체 생성
        class LaunchResult:
            def __init__(self, arn, runtime_id):
                self.agent_arn = arn
                self.runtime_id = runtime_id

        launch_result = LaunchResult(agent_arn, runtime_id)
    else:
        print("❌ Error creating Runtime:")
        print(result.stderr)
        raise Exception(f"Runtime creation failed: {result.stderr}")
finally:
    # 임시 파일 삭제
    import os
    if os.path.exists(json_file):
        os.unlink(json_file)

⚠️  Using PUBLIC runtime's ECR image: 057716757052.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-bedrock_manus_runtime
Creating VPC Runtime with AWS CLI v2
ECR Image: 057716757052.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-bedrock_manus_runtime:latest
VPC Subnets: ['subnet-0b2fb367d6e823a79']
Security Groups: ['sg-0affaea9ac4dc26b1']

🚀 Creating Runtime: bedrock_manus_runtime_vpc_1760773105

✅ Runtime Created!
   ARN: arn:aws:bedrock-agentcore:us-east-1:057716757052:runtime/bedrock_manus_runtime_vpc_1760773105-PtBWr17D4z
   ID: bedrock_manus_runtime_vpc_1760773105-PtBWr17D4z
   Status: CREATING

ℹ️  Note: Runtime created in VPC mode with networkModeConfig


## Step 5. 배포 상태 확인: `agentcore_runtime.status()`

In [28]:
import time
import boto3

# Runtime ID 추출
runtime_id = launch_result.runtime_id
agentcore_control = boto3.client('bedrock-agentcore-control', region_name=region)

print(f"⏳ Checking Runtime status: {runtime_id}")
print()

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
status = 'CREATING'

while status not in end_status:
    time.sleep(10)
    
    try:
        response = agentcore_control.get_agent_runtime(agentRuntimeId=runtime_id)
        status = response['status']
        print(f"  Status: {status}")
    except Exception as e:
        print(f"  Error checking status: {e}")
        break

print()
if status == 'READY':
    print("=" * 60)
    print("✅ Runtime is READY!")
    print("=" * 60)
    print()
    print("🔒 VPC Private connectivity is active")
    print("   All traffic will go through VPC Endpoint (no public internet)")
    print()
    
    # Observability 설정 검증
    print("🔍 Verifying observability configuration...")
    
    runtime_details = agentcore_control.get_agent_runtime(agentRuntimeId=runtime_id)
    
    obs_config = runtime_details.get('observabilityConfiguration')
    if obs_config and obs_config.get('cloudWatch', {}).get('enabled'):
        log_group = obs_config['cloudWatch'].get('logGroup', 'N/A')
        print("  ✅ Observability: ENABLED")
        print(f"     CloudWatch Log Group: {log_group}")
        print(f"     You can now debug health check issues via CloudWatch logs!")
    else:
        print("  ⚠️  Observability: NOT ENABLED")
        print("     WARNING: Cannot debug health check issues without logs!")
    
    print()
    print("=" * 60)
    print("🎯 Next Steps:")
    print("=" * 60)
    print("1. Wait 1-2 minutes for container to fully start")
    print("2. Check CloudWatch logs to see container startup")
    print("3. Test Runtime invocation in next cell")
    print("4. If health check fails, check logs for the reason")
    print("=" * 60)
else:
    print(f"❌ Runtime status: {status}")
    print("   Please check CloudWatch logs for details")
    print(f"   Runtime ID: {runtime_id}")

print()
print(f"Final Status: {status}")

⏳ Checking Runtime status: bedrock_manus_runtime_vpc_1760773105-PtBWr17D4z

  Status: READY

✅ Runtime is READY!

🔒 VPC Private connectivity is active
   All traffic will go through VPC Endpoint (no public internet)

🔍 Verifying observability configuration...
  ⚠️  Observability: NOT ENABLED

🎯 Next Steps:
1. Wait 1-2 minutes for container to fully start
2. Check CloudWatch logs to see container startup
3. Test Runtime invocation in next cell
4. If health check fails, check logs for the reason

Final Status: READY


---

# 2) 배포한 에이전트 Health Check


## 에이전트 호출

In [29]:
# Runtime 호출 테스트
import boto3
import json

agentcore_client = boto3.client('bedrock-agentcore', region_name=region)

print("🧪 Testing VPC Runtime health...")
print(f"   Runtime ID: {runtime_id}")
print()

try:
    response = agentcore_client.invoke_agent_runtime(
        agentRuntimeArn=launch_result.agent_arn,
        qualifier="DEFAULT",
        payload=json.dumps({"prompt": "Hello, test message"})
    )

    print("✅ Health check PASSED!")
    print("   Runtime is responding correctly")

except Exception as e:
    print("❌ Health check FAILED!")
    print(f"   Error: {str(e)}")
    print()
    print("⚠️  This is the same issue as previous VPC Runtimes")
    print("   Without observability, we cannot see container logs")

🧪 Testing VPC Runtime health...
   Runtime ID: bedrock_manus_runtime_vpc_1760773105-PtBWr17D4z

✅ Health check PASSED!
   Runtime is responding correctly
